In [1]:
import numpy
import pandas
import tensorflow
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold  
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from tensorflow.keras import models,layers,utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,Activation,Flatten,Bidirectional,Conv1D,MaxPooling1D,LeakyReLU
import os
from tcn import TCN,tcn_full_summary
from tensorflow.keras import Input

In [2]:
# MAPE SMAPE
from numpy import mean
def maped(y_true, y_pred):
  return mean(abs((y_pred - y_true) / y_true)) * 100
def smape(y_true, y_pred):
  return 2.0 * mean(abs(y_pred - y_true) / (abs(y_pred) + abs(y_true))) * 100

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
physical_gpus = tensorflow.config.list_physical_devices("GPU")
tensorflow.config.experimental.set_virtual_device_configuration(
    physical_gpus[0],
    [tensorflow.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)]
)
logical_gpus = tensorflow.config.list_logical_devices("GPU")

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# load dataset
dataset = read_csv(r"\20220511\20220418.csv", header=0, index_col=0)
values = dataset.values
# ensure all data is float
values = values.astype('float32')
max_value = max(values[:,-1])
min_value = min(values[:,-1])
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 0, 1)

In [ ]:
# split into train and test sets
values = reframed.values
n_train = int(values.shape[0])-4740
n_validation = int(values.shape[0])-3950
n_test1 = int(values.shape[0])-3160

train = values[:n_train, :]
validation = values[n_train:n_validation, :]
test1 = values[n_validation:n_test1, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
validation_X, validation_y = validation[:, :-1], validation[:, -1]
test_X1, test_y1 = test1[:, :-1], test1[:, -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, 7))
validation_X = validation_X.reshape((validation_X.shape[0], 1, 7))
test_X1 = test_X1.reshape((test_X1.shape[0], 1, 7))

In [8]:
# design network
batch_size = None
model = Sequential()
input_layer = Input(batch_shape=(batch_size,1,7))
model.add(input_layer)
model.add(TCN(nb_filters=100, #Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM. Can be a list.
        kernel_size=3, #Integer. The size of the kernel to use in each convolutional layer.
        nb_stacks=1,   #The number of stacks of residual blocks to use.
        dilations=(1,2,4), #List/Tuple. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
        padding='causal',
        use_skip_connections=False, 
        dropout_rate=0.1,
        return_sequences=False,
        activation='relu', 
        kernel_initializer='he_normal', 
        use_batch_norm=False, 
        use_layer_norm=False, 
        ))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(16))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(1))
model.add(LeakyReLU(alpha=0.3))
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn_2 (TCN)                  (None, 100)               153500    
_________________________________________________________________
dense_8 (Dense)              (None, 64)                6464      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                528       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 16)               

In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=250,batch_size=250, validation_data=(validation_X, validation_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='validation')
pyplot.legend()
pyplot.show()